#Teknologi Perekayasaan Data
##Modul 4
###Rambati Lumbung Pardja
###222212833
###3SI1/31

####PENUGASAN (1 Week Assignment)
1. Buat File .ipynb yang baru dengan nama Praktikum4_NIM.ipynb

2. Lakukan web scraping menggunakan Requests + BeautifulSoup untuk mengekstrak Tabel Matching Cryptocurrencies pada tab Table View di Yahoo!Finance https://finance.yahoo.com/crypto/ (cukup 3 halaman saja - 75 rows). Hints: lihat perubahan URL untuk setiap halaman table dan lakukan looping per halaman

Persiapan

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

FUngsi scraping

In [2]:
def scrape_crypto_data(page_count=3):
    base_url = "https://finance.yahoo.com/crypto/?offset={}&count=25"
    crypto_data = []

    for page in range(page_count):
        offset = page * 25
        url = base_url.format(offset)
        headers = {"User-Agent": "Mozilla/5.0"}

        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            table = soup.find("table")

            if table:
                rows = table.find_all("tr")[1:]

                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 7:

                        # buat debug
                        col_texts = [col.text.strip() for col in cols]
                        print("Row Data:", col_texts)

                        name = cols[0].text.strip()
                        symbol = cols[1].text.strip()
                        price_span = cols[2].find("span")
                        price = price_span.text.strip() if price_span else "N/A"
                        change_span = cols[3].find("span")
                        change = change_span.text.strip() if change_span else "N/A"
                        percent_change_span = cols[4].find("span")
                        percent_change = percent_change_span.text.strip() if percent_change_span else "N/A"
                        market_cap_span = cols[6].find("span")
                        market_cap = market_cap_span.text.strip() if market_cap_span else "N/A"

                        crypto_data.append([symbol, name, price, change, percent_change, market_cap])

            else:
                print(f"Tabel tidak ditemukan pada halaman {page + 1}")
        else:
            print(f"Gagal mengakses halaman {page + 1}")

    return crypto_data

fungsi simpan ke csv

In [3]:
def save_to_csv(data, filename="crypto_data.csv"):
    df = pd.DataFrame(data, columns=["Symbol", "Name", "Price", "Change", "% Change", "Market Cap"])
    df.to_csv(filename, index=False)
    print(f"Data berhasil disimpan ke {filename}")

main function

In [4]:
def main():
    print("Sedang mencuri data dari Yahoo! Finance...3...2...1...")
    crypto_data = scrape_crypto_data(page_count=3)

    if crypto_data:
        save_to_csv(crypto_data)
        print("\n5 Data pertama yang berhasil didapat:")
        print(pd.DataFrame(crypto_data, columns=["Name", "Symbol", "Price", "Change", "% Change", "Market Cap"]).head())
    else:
        print("Enggak ada data yang diambil")

if __name__ == "__main__":
    main()

🛠️ Sedang mencuri data dari Yahoo! Finance...3...2...1...
Row Data: ['BTC-USD', 'Bitcoin USD', '', '85,182.27 +3,104.50 (+3.78%)', '+3,104.50', '+3.78%', '1.69T', '30.978B', '30.978B', '30.978B', '19.839M', '21.80%', '']
Row Data: ['ETH-USD', 'Ethereum USD', '', '2,028.66 +130.80 (+6.89%)', '+130.80', '+6.89%', '244.711B', '18.548B', '18.548B', '18.548B', '120.627M', '-44.99%', '']
Row Data: ['XRP-USD', 'XRP USD', '', '2.4949 +0.2410 (+10.69%)', '+0.2410', '+10.69%', '144.976B', '8.752B', '8.752B', '8.752B', '58.109B', '273.59%', '']
Row Data: ['USDT-USD', 'Tether USDt USD', '', '1.0003 +0.0007 (+0.07%)', '+0.0007', '+0.07%', '143.488B', '72.9B', '72.9B', '72.9B', '143.439B', '-0.05%', '']
Row Data: ['BNB-USD', 'BNB USD', '', '617.62 -14.73 (-2.33%)', '-14.73', '-2.33%', '87.995B', '1.985B', '1.985B', '1.985B', '142.474M', '12.83%', '']
Row Data: ['SOL-USD', 'Solana USD', '', '131.96 +7.96 (+6.42%)', '+7.96', '+6.42%', '67.305B', '3.195B', '3.195B', '3.195B', '510.033M', '-34.58%', '']

3. Lakukan web scraping menggunakan Scrapy untuk mengekstrak data negara di https://www.scrapethissite.com/pages/simple/

In [12]:
!pip install scrapy

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

class CountriesSpider(scrapy.Spider):
    name = "countries"
    start_urls = ["https://www.scrapethissite.com/pages/simple/"]

    def parse(self, response):
        countries = response.css("div.country")

        for country in countries:
            yield {
                # pake string() buat ambil text dari <h3>
                "name": country.xpath("string(.//h3[@class='country-name'])").get(default="").strip(),
                "capital": country.css("span.country-capital::text").get(default="").strip(),
                "population": country.css("span.country-population::text").get(default="").strip(),
                "area": country.css("span.country-area::text").get(default="").strip(),
            }

# buat jalaninnya
process = CrawlerProcess(settings={
    "FEEDS": {
        "countries.json": {
            "format": "json",
            "indent": 4,
        }
    }
})

process.crawl(CountriesSpider)
process.start()


INFO:scrapy.utils.log:Scrapy 2.12.0 started (bot: scrapybot)
2025-03-20 02:40:37 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2025-03-20 02:40:37 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.12.9, cssselect 1.3.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2025-03-20 02:40:37 [scrapy.addons] INFO: Enabled addons:
[]
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2025-03-20 02:40:37 [scrapy.utils.log] DEBUG: Using reactor: twi

In [ ]:
import json

data = []
with open("countries.json", "r") as file:
    for line in file:
        try:
            data.append(json.loads(line))  # Load tiap baris JSON
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

# Cek 5 data pertama
print("\n5 Data Negara yang Berhasil Diambil:")
for country in data[:5]:
    print(country)


4. Buat data scraping menggunakan API:

- Pilih salah satu penyedia data: Twitter/Instagram/Facebook/Google/atau lainnya
- Sign Up untuk mendapatkan akses autentikasi API (jika diperlukan)
- Baca dan pelajari dokumentasi API
- Ambil data dari penyedia data (cukup satu jenis data saja - 1 API method)

In [9]:
import requests
import time

API_KEY = "fb84a884"

# buat parameternya
directors_movies = {
    "Christopher Nolan": ["Inception", "Interstellar", "The Dark Knight", "Memento", "Dunkirk"],
    "Quentin Tarantino": ["Pulp Fiction", "Django Unchained", "Kill Bill: Vol. 1", "Inglourious Basterds", "Once Upon a Time in Hollywood"],
    "Martin Scorsese": ["Goodfellas", "The Wolf of Wall Street", "Taxi Driver", "The Irishman", "Casino"],
    "Steven Spielberg": ["Jurassic Park", "Schindler's List", "E.T.", "Jaws", "Indiana Jones"],
    "Ridley Scott": ["Gladiator", "Blade Runner", "The Martian", "Alien", "Black Hawk Down"],
    "James Cameron": ["Titanic", "Avatar", "Terminator 2", "Aliens", "The Abyss"],
    "David Fincher": ["Fight Club", "Se7en", "Gone Girl", "The Social Network", "Zodiac"],
    "Stanley Kubrick": ["The Shining", "A Clockwork Orange", "2001: A Space Odyssey", "Full Metal Jacket", "Dr. Strangelove"],
    "Peter Jackson": ["The Lord of the Rings: The Fellowship of the Ring", "The Hobbit", "King Kong", "The Lovely Bones", "The Frighteners"],
    "Denis Villeneuve": ["Dune", "Arrival", "Blade Runner 2049", "Prisoners", "Sicario"],

    "Joko Anwar": ["Pengabdi Setan", "Gundala", "A Copy of My Mind", "Modus Anomali", "Janji Joni"],
    "Hanung Bramantyo": ["Ayat-Ayat Cinta", "Perempuan Berkalung Sorban", "Kartini", "Sang Pencerah", "Rudy Habibie"],
    "Ernest Prakasa": ["Cek Toko Sebelah", "Imperfect", "Ngenest", "Susah Sinyal", "Milly & Mamet"],
    "Rizal Mantovani": ["Jelangkung", "Wewe", "Kuntilanak", "5 cm", "Air Terjun Pengantin"],
    "Upi Avianto": ["My Stupid Boss", "Realita Cinta dan Rock n Roll", "Radit dan Jani", "Serigala Terakhir", "Belenggu"],
    "Mouly Surya": ["Marlina si Pembunuh dalam Empat Babak", "Fiksi.", "What They Don't Talk About When They Talk About Love"],
    "Angga Dwimas Sasongko": ["Filosofi Kopi", "Surat dari Praha", "Nanti Kita Cerita tentang Hari Ini", "Cahaya dari Timur"],
    "Garin Nugroho": ["Kucumbu Tubuh Indahku", "Opera Jawa", "Daun di Atas Bantal", "Soegija", "Laskar Pelangi 2"],
    "Riri Riza": ["Ada Apa dengan Cinta?", "Laskar Pelangi", "Athirah", "Sokola Rimba", "Gie"],
    "Teddy Soeriaatmadja": ["Lovely Man", "Something in the Way", "About a Woman", "Banyu Biru"],
}

# buat nyimpen hasilnya
movies_data = []

In [10]:
for director, movies in directors_movies.items():
    print(f"Mencuri data Absolute Cinema dari: {director}")
    for title in movies:
        url = f"https://www.omdbapi.com/?t={title}&apikey={API_KEY}"
        response = requests.get(url)
        data = response.json()

        if data.get("Response") == "True":
            movies_data.append({
                "Title": data.get("Title"),
                "Year": data.get("Year"),
                "Director": director,
                "Genre": data.get("Genre"),
                "IMDB Rating": data.get("imdbRating"),
                "Plot": data.get("Plot"),
            })
            print(f"{title} berhasil diambil.")
        else:
            print(f"Film '{title}' gak ada, gan.")

        time.sleep(1)  # Hindari rate limit API

# Simpan hasilnya ke file JSON
import json
with open("movies_data.json", "w", encoding="utf-8") as f:
    json.dump(movies_data, f, indent=4)

print("\nData berhasil dicuri! Cek di 'movies_data.json'")


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.omdbapi.com:443
2025-03-20 01:24:11 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.omdbapi.com:443
DEBUG:urllib3.connectionpool:https://www.omdbapi.com:443 "GET /?t=Inception&apikey=fb84a884 HTTP/1.1" 200 None
2025-03-20 01:24:11 [urllib3.connectionpool] DEBUG: https://www.omdbapi.com:443 "GET /?t=Inception&apikey=fb84a884 HTTP/1.1" 200 None


📌 Mengambil data film dari: Christopher Nolan
✅ Inception berhasil diambil.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.omdbapi.com:443
2025-03-20 01:24:12 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.omdbapi.com:443
DEBUG:urllib3.connectionpool:https://www.omdbapi.com:443 "GET /?t=Interstellar&apikey=fb84a884 HTTP/1.1" 401 47
2025-03-20 01:24:13 [urllib3.connectionpool] DEBUG: https://www.omdbapi.com:443 "GET /?t=Interstellar&apikey=fb84a884 HTTP/1.1" 401 47


⚠️ Film 'Interstellar' tidak ditemukan!


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.omdbapi.com:443
2025-03-20 01:24:14 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.omdbapi.com:443
DEBUG:urllib3.connectionpool:https://www.omdbapi.com:443 "GET /?t=The%20Dark%20Knight&apikey=fb84a884 HTTP/1.1" 200 None
2025-03-20 01:24:14 [urllib3.connectionpool] DEBUG: https://www.omdbapi.com:443 "GET /?t=The%20Dark%20Knight&apikey=fb84a884 HTTP/1.1" 200 None


...

✅ Something in the Way berhasil diambil.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.omdbapi.com:443
2025-03-20 01:26:00 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.omdbapi.com:443
DEBUG:urllib3.connectionpool:https://www.omdbapi.com:443 "GET /?t=About%20a%20Woman&apikey=fb84a884 HTTP/1.1" 200 None
2025-03-20 01:26:00 [urllib3.connectionpool] DEBUG: https://www.omdbapi.com:443 "GET /?t=About%20a%20Woman&apikey=fb84a884 HTTP/1.1" 200 None


✅ About a Woman berhasil diambil.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.omdbapi.com:443
2025-03-20 01:26:01 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.omdbapi.com:443
DEBUG:urllib3.connectionpool:https://www.omdbapi.com:443 "GET /?t=Banyu%20Biru&apikey=fb84a884 HTTP/1.1" 200 None
2025-03-20 01:26:01 [urllib3.connectionpool] DEBUG: https://www.omdbapi.com:443 "GET /?t=Banyu%20Biru&apikey=fb84a884 HTTP/1.1" 200 None


✅ Banyu Biru berhasil diambil.

🎉 Data scraping selesai! Hasilnya disimpan di 'movies_data.json'


In [11]:
print("\nHasil mencuri data Absoulute Cinema:")
for movie in movies_data:
    print("-" * 40)
    for key, value in movie.items():
        print(f"{key}: {value}")


🎬 Hasil Scraping Film:
----------------------------------------
Title: Inception
Year: 2010
Director: Christopher Nolan
Genre: Action, Adventure, Sci-Fi
IMDB Rating: 8.8
Plot: A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.
----------------------------------------
Title: The Dark Knight
Year: 2008
Director: Christopher Nolan
Genre: Action, Crime, Drama
IMDB Rating: 9.0
Plot: When a menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman, James Gordon and Harvey Dent must work together to put an end to the madness.
----------------------------------------
Title: Dunkirk
Year: 2017
Director: Christopher Nolan
Genre: Action, Drama, History
IMDB Rating: 7.8
Plot: Allied soldiers from Belgium, the British Commonwealth and Empire, and France are surrounded by the German Army and evacuated during 